In [13]:
import pandas as pd
from keras.models import load_model
import numpy as np
pdg_dict={'p':2212,'mu-':13, 'gamma':22, 'pi0':11,'pi+':211, 'e':11 }

In [14]:
event_df = pd.read_csv('../DL_data/1004nuance_mu_p_gamma_gamma.csv', index_col=0).tail(373*2)
event_df2 = pd.read_csv('../DL_data/1003nuance_mu_p_pi+.csv', index_col=0).tail(2000)
event_df2.head(2)

,index,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event,x_scaled,...,774,775,776,777,778,779,780,781,782,783
2875,69168,1,1003,2212,0.981818,1.0,"[24.2038, 24.2778, 23.7412, 23.544, 23.3459, 2...","[19.45, 19.75, 20.05, 20.35, 20.65, 20.95, 21....",29916,[ 27. 27. 25. 24. 23. 22. 21. 20. 19. ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,255.0
2876,69169,1,1003,211,0.971831,1.0,"[25.2876, 25.8769, 27.3528, 26.4325, 27.5133, ...","[20.05, 20.35, 20.65, 20.65, 20.95, 20.95, 21....",29916,[ 0. 1. 4. 2. 5. 3. 5. 6. 7. ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
pixel_cols=[str(i) for i in range(28*28)]

In [16]:
model = load_model('my_model_1.h5')

In [17]:
event_1_test = (event_df[pixel_cols].values).astype('float32')
event_2_test = (event_df2[pixel_cols].values).astype('float32')

In [18]:
event_1_test=event_1_test/255.0
event_2_test=event_2_test/255.0
event_1_test = event_1_test.reshape(event_1_test.shape[0], 28, 28,1)
event_2_test = event_2_test.reshape(event_2_test.shape[0], 28, 28,1)
event_2_test

array([[[[ 1.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 1.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        ..., 
        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 1.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 1.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 1.]]],


       [[[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 1.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 1.],
         [ 0.],
         [ 1.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
 

In [19]:
# Predict the values from the validation dataset
y_pred = model.predict(event_1_test)
y_pred = np.argmax(y_pred, axis = 1) 
event_df['track_pred']=y_pred
event_df.head(2)

,index,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event,x_scaled,...,775,776,777,778,779,780,781,782,783,track_pred
746,55211,1,1004,2212,1.0,0.934783,"[151.193, 151.231, 151.251, 151.276, 151.296, ...","[548.05, 548.35, 548.65, 548.95, 549.25, 549.5...",23855,[ 0. 0. 0. 0. 0. 1. 1. 1. 1. ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
747,55212,1,1004,13,1.0,0.975845,"[151.16, 151.217, 151.303, 151.425, 151.571, 1...","[546.55, 546.85, 547.15, 547.45, 547.75, 548.0...",23855,[ 0. 0. 0. 0. 1. 1. 1. 1. 2. ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [20]:
particles = ['mu-', 'p', 'gamma']
accuracy=[]
for pdg in particles:
    true = len(event_df[event_df.pdg==pdg_dict[pdg]])
    pred = len(event_df[(event_df.isTrueTrack==event_df.track_pred)][event_df.pdg==pdg_dict[pdg]])
    accuracy.append(pred/true)
print('CC_Nu_Mu nuance code:1004')
accuracy = [[i] for i in accuracy]
dictionary = dict(zip(particles, accuracy))
result = pd.DataFrame(data=dictionary)
result['prediction accuracy']= '%'
result =result.set_index('prediction accuracy')
result=result*100
result

CC_Nu_Mu nuance code:1004


C:\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,gamma,mu-,p
prediction accuracy,,,
%,78.494624,97.860963,93.582888


In [21]:
y_pred = model.predict(event_2_test)
y_pred = np.argmax(y_pred, axis = 1) 
event_df2['track_pred']=y_pred
event_df2.head(2)

,index,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event,x_scaled,...,775,776,777,778,779,780,781,782,783,track_pred
2875,69168,1,1003,2212,0.981818,1.0,"[24.2038, 24.2778, 23.7412, 23.544, 23.3459, 2...","[19.45, 19.75, 20.05, 20.35, 20.65, 20.95, 21....",29916,[ 27. 27. 25. 24. 23. 22. 21. 20. 19. ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,255.0,1
2876,69169,1,1003,211,0.971831,1.0,"[25.2876, 25.8769, 27.3528, 26.4325, 27.5133, ...","[20.05, 20.35, 20.65, 20.65, 20.95, 20.95, 21....",29916,[ 0. 1. 4. 2. 5. 3. 5. 6. 7. ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [22]:
particles = ['mu-', 'p', 'pi+']
accuracy=[]
for pdg in particles:
    true = len(event_df2[event_df2.pdg==pdg_dict[pdg]])
    pred = len(event_df2[(event_df2.isTrueTrack==event_df2.track_pred)][event_df2.pdg==pdg_dict[pdg]])
    try:
        accuracy.append(pred/true)
    except:
        accuracy.append(0)
print('CC_Nu_Mu nuance code:1004')
accuracy = [[i] for i in accuracy]
dictionary = dict(zip(particles, accuracy))
result = pd.DataFrame(data=dictionary)
result['prediction accuracy']= '%'
result =result.set_index('prediction accuracy')
result=result*100
result

CC_Nu_Mu nuance code:1004


C:\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,mu-,p,pi+
prediction accuracy,,,
%,93.543544,93.403298,83.058471
